In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2022 data into respective dataframes
df1 = pd.read_csv('../Data/2022/202201-divvy-tripdata.csv')
df2 = pd.read_csv('../Data/2022/202202-divvy-tripdata.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665968,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


In [4]:
dfbike.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

### Saving all the station names in a list "allSt22"

#### Comparing with 2021 data and only adding the stations which are new into a list "added22"
#### As this year has only two months of data, all the stations might not have been used already because it was January and February and the usage is usally less during this time. Hence, not considering removed stations

In [5]:
allSt22 = list(dfbike['start_station_name'].unique())
len(allSt22)

808

In [6]:
allSt21 = ['California Ave & Cortez St',
 'Halsted St & North Branch St',
 'Glenwood Ave & Touhy Ave',
 'Michigan Ave & 8th St',
 'Shedd Aquarium',
 'Sedgwick St & North Ave',
 'Rush St & Hubbard St',
 'Campbell Ave & Montrose Ave',
 'State St & Harrison St',
 'Sheffield Ave & Fullerton Ave',
 'State St & Pearson St',
 'Damen Ave & Leland Ave',
 '2112 W Peterson Ave',
 'St. Clair St & Erie St',
 'Southport Ave & Wellington Ave',
 'Indiana Ave & Roosevelt Rd',
 'Lincoln Park Conservatory',
 'Clark St & Armitage Ave',
 'Wood St & Augusta Blvd',
 'Calumet Ave & 35th St',
 'Halsted St & Roscoe St',
 'Wood St & Chicago Ave',
 'Kedzie Ave & Milwaukee Ave',
 'Larrabee St & Division St',
 'Damen Ave & Cortland St',
 'Halsted St & Archer Ave',
 'Honore St & Division St',
 'LaSalle Dr & Huron St',
 'Lincoln Ave & Addison St',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Ogden Ave & Chicago Ave',
 'Drake Ave & Montrose Ave',
 'Ritchie Ct & Banks St',
 'Clarendon Ave & Junior Ter',
 'Orleans St & Hubbard St',
 'Rush St & Superior St',
 'McCormick Place',
 'Lincoln Ave & Waveland Ave',
 'Franklin St & Monroe St',
 'Franklin St & Jackson Blvd',
 'Fairbanks St & Superior St',
 'Halsted St & Maxwell St',
 'Damen Ave & Cullerton St',
 'Michigan Ave & 14th St',
 'Noble St & Milwaukee Ave',
 'Oakley Ave & Irving Park Rd',
 'Indiana Ave & 26th St',
 'Greenview Ave & Fullerton Ave',
 'Loomis St & Jackson Blvd',
 'Washtenaw Ave & Ogden Ave',
 'Benson Ave & Church St',
 'University Ave & 57th St',
 'Logan Blvd & Elston Ave',
 'Milwaukee Ave & Rockwell St',
 'Desplaines St & Jackson Blvd',
 'Orleans St & Merchandise Mart Plaza',
 'Michigan Ave & Jackson Blvd',
 'Halsted St & Clybourn Ave',
 'Rhodes Ave & 32nd St',
 'Marine Dr & Ainslie St',
 'Troy St & North Ave',
 'Racine Ave & 15th St',
 'Wolcott Ave & Fargo Ave',
 'Clark St & Chicago Ave',
 'Wells St & Polk St',
 'Clark St & Wrightwood Ave',
 'Ashland Ave & Wellington Ave',
 'Damen Ave & Sunnyside Ave',
 'Washtenaw Ave & Lawrence Ave',
 'Paulina St & Montrose Ave',
 'Wells St & Elm St',
 'New St & Illinois St',
 'Clinton St & Jackson Blvd',
 'Federal St & Polk St',
 'California Ave & Francis Pl (Temp)',
 'Christiana Ave & Lawrence Ave',
 'Racine Ave & 13th St',
 'Warren Park West',
 'Wolcott (Ravenswood) Ave & Montrose Ave',
 'Leavitt St & Armitage Ave',
 'Clark St & Elmdale Ave',
 'Wood St & 35th St',
 'Clark St & Berwyn Ave',
 'Southport Ave & Wrightwood Ave',
 'Wentworth Ave & 24th St (Temp)',
 'Southport Ave & Roscoe St',
 'California Ave & Fletcher St',
 'Damen Ave & Walnut (Lake) St',
 'Western Ave & Division St',
 'Wentworth Ave & 63rd St',
 'Pulaski Rd & Eddy St (Temp)',
 'Dearborn Pkwy & Delaware Pl',
 'California Ave & North Ave',
 'Richmond St & Diversey Ave',
 'Halsted St & 35th St',
 'Shields Ave & 31st St',
 'Racine Ave & Congress Pkwy',
 'Calumet Ave & 33rd St',
 'Pine Grove Ave & Irving Park Rd',
 'Wells St & Huron St',
 'Fort Dearborn Dr & 31st St',
 'Paulina St & Howard St',
 'Central Park Ave & Elbridge Ave',
 'Wells St & Evergreen Ave',
 'Lakeview Ave & Fullerton Pkwy',
 'Kilbourn Ave & Irving Park Rd',
 'St. Louis Ave & Balmoral Ave',
 'Broadway & Belmont Ave',
 'Lincoln Ave & Winona St',
 'Clark St & Drummond Pl',
 'Wilton Ave & Diversey Pkwy',
 'May St & Cullerton St',
 'Clinton St & 18th St',
 'Lake Shore Dr & Belmont Ave',
 'Greenwood Ave & 47th St',
 'Western Ave & 24th St',
 'Lincoln Ave & Fullerton Ave',
 'Wilton Ave & Belmont Ave',
 'Broadway & Sheridan Rd',
 'Emerald Ave & 28th St',
 'Halsted St & Willow St',
 'Clinton St & Polk St',
 'Wabash Ave & 9th St',
 'Paulina St & Flournoy St',
 'Racine Ave & 35th St',
 'Normal Ave & Archer Ave',
 'Clark St & Columbia Ave',
 'Larrabee St & Menomonee St',
 'Clark St & Lake St',
 'Damen Ave & Clybourn Ave',
 'Sheffield Ave & Webster Ave',
 'Larrabee St & Oak St',
 'Theater on the Lake',
 'California Ave & Lake St',
 'Dorchester Ave & 49th St',
 'Smith Park',
 'Woodlawn Ave & 55th St',
 'Sangamon St & Washington Blvd',
 'Keystone Ave & Fullerton Ave',
 'Kildare Ave & Montrose Ave',
 'Lake Shore Dr & Wellington Ave',
 'Clark St & Elm St',
 'Princeton Ave & 47th St',
 'Sheffield Ave & Willow St',
 'Elston Ave & Wabansia Ave',
 'Michigan Ave & Lake St',
 'Mies van der Rohe Way & Chicago Ave',
 'Spaulding Ave & Division St',
 'Wells St & Hubbard St',
 'Clark St & Schiller St',
 'Kingsbury St & Erie St',
 'Larrabee St & North Ave',
 'California Ave & Milwaukee Ave',
 'Damen Ave & Pierce Ave',
 'Throop St & Taylor St',
 'Paulina Ave & North Ave',
 'Bissell St & Armitage Ave',
 'Central Ave & Madison St',
 'Southport Ave & Irving Park Rd',
 'Damen Ave & Division St',
 'Lake Shore Dr & Diversey Pkwy',
 'Bosworth Ave & Howard St',
 'Valli Produce - Evanston Plaza',
 'Campbell Ave & Fullerton Ave',
 'Damen Ave & Foster Ave',
 'Kimball Ave & Belmont Ave',
 'Racine Ave & Washington Blvd',
 'Wood St & Hubbard St',
 'Financial Pl & Ida B Wells Dr',
 'Canal St & Adams St',
 'University Library (NU)',
 'Albany Ave & 26th St',
 'MLK Jr Dr & 56th St',
 'Evanston Civic Center',
 'Dodge Ave & Main St',
 'Ashland Ave & Archer Ave',
 'Ridge Blvd & Touhy Ave',
 'Clifton Ave & Armitage Ave',
 'Artesian Ave & Hubbard St',
 'Wacker Dr & Washington St',
 'Halsted St & 21st St',
 'Peoria St & Jackson Blvd',
 'Sedgwick St & Webster Ave',
 'Broadway & Thorndale Ave',
 'Ashland Ave & Belle Plaine Ave',
 'Canal St & Taylor St',
 'LaSalle St & Jackson Blvd',
 'Morgan St & Lake St',
 'Columbus Dr & Randolph St',
 'State St & Randolph St',
 'Michigan Ave & Ida B Wells Dr',
 'Broadway & Cornelia Ave',
 'State St & Kinzie St',
 'Dearborn St & Adams St',
 'Cottage Grove Ave & Oakwood Blvd',
 'Leavitt St & Chicago Ave',
 'Wentworth Ave & 35th St',
 'Western Ave & 21st St',
 'Winthrop Ave & Lawrence Ave',
 'Montrose Harbor',
 'Ellis Ave & 53rd St',
 'Streeter Dr & Grand Ave',
 'Wood St & Milwaukee Ave',
 'Morgan St & Polk St',
 'Halsted St & Polk St',
 'Maplewood Ave & Peterson Ave',
 'Lake Park Ave & 56th St',
 'Damen Ave & Charleston St',
 'State St & 35th St',
 'Buckingham Fountain',
 'LaSalle St & Adams St',
 'Central Ave & Lake St',
 'Jefferson St & Monroe St',
 'Wabash Ave & Roosevelt Rd',
 'Halsted St & 69th St',
 'Latrobe Ave & Chicago Ave',
 'Milwaukee Ave & Wabansia Ave',
 'Sedgwick St & Huron St',
 'Ashland Ave & Grand Ave',
 'Campbell Ave & North Ave',
 'California Ave & Altgeld St',
 'Clarendon Ave & Leland Ave',
 'DuSable Museum',
 'Southport Ave & Clark St',
 'LaSalle St & Washington St',
 'Albany Ave & Montrose Ave',
 'Spaulding Ave & Armitage Ave',
 'Clark St & Bryn Mawr Ave',
 'California Ave & Byron St',
 'McClurg Ct & Erie St',
 '900 W Harrison St',
 'Halsted St & Dickens Ave',
 'Indiana Ave & 31st St',
 'Wabash Ave & Grand Ave',
 'Wabash Ave & Cermak Rd',
 'Racine Ave & 18th St',
 'Franklin St & Chicago Ave',
 'Clark St & Randolph St',
 'Mies van der Rohe Way & Chestnut St',
 'Perry Ave & 69th St',
 'Larrabee St & Kingsbury St',
 'Clinton St & Roosevelt Rd',
 'Western Ave & Lunt Ave',
 'Damen Ave & Chicago Ave',
 'Damen Ave & Grand Ave',
 'Central Park Ave & Ogden Ave',
 'Kedzie Ave & Chicago Ave',
 'Troy St & Elston Ave',
 'Lincoln Ave & Sunnyside Ave',
 'Kimbark Ave & 53rd St',
 'Aberdeen St & Monroe St',
 'Green St & Madison St',
 'Ashland Ave & Blackhawk St',
 'Clinton St & Lake St',
 'Ada St & Washington Blvd',
 'Drake Ave & Addison St',
 'Clark St & Jarvis Ave',
 'Chicago Ave & Washington St',
 'Clark St & Schreiber Ave',
 'Clark St & Montrose Ave',
 'Greenview Ave & Jarvis Ave',
 'MLK Jr Dr & 63rd St',
 'Greenwood Ave & 79th St',
 'Ashland Ave & Wrightwood Ave',
 'Racine Ave & Belmont Ave',
 'Canal St & Harrison St',
 'Clark St & 9th St (AMLI)',
 'Emerald Ave & 31st St',
 'Stockton Dr & Wrightwood Ave',
 'Lincoln Ave & Belmont Ave',
 'Clinton St & Madison St',
 'Lake Park Ave & 35th St',
 'Kedzie Ave & Bryn Mawr Ave',
 'Leavitt St & Belmont Ave',
 'Hoyne Ave & Balmoral Ave',
 'Ridge Blvd & Howard St',
 'Racine Ave & Garfield Blvd',
 'Vernon Ave & 79th St',
 'Kosciuszko Park',
 'Hegewisch Metra Station',
 'Ashland Ave & 63rd St',
 'Damen Ave & Madison St',
 'Jeffery Blvd & 71st St',
 'Monticello Ave & Irving Park Rd',
 'Ashland Ave & 78th St',
 'Oglesby Ave & 100th St',
 'Archer (Damen) Ave & 37th St',
 'Burnham Greenway & 105th St',
 'Ashland Ave & Lake St',
 'Hoyne Ave & 47th St',
 'Humboldt Blvd & Armitage Ave',
 'Conservatory Dr & Lake St',
 'Central St & Girard Ave',
 'Millard Ave & 26th St',
 'Halsted St & 47th Pl',
 'Museum of Science and Industry',
 'Kedzie Ave & Palmer Ct',
 'Ellis Ave & 55th St',
 'Harper Ave & 59th St',
 'Kedzie Ave & Lake St',
 'Western Ave & Walton St',
 'Western Ave & Fillmore St',
 'Commercial Ave & 100th St',
 'Claremont Ave & Hirsch St',
 'Clark St & Ida B Wells Dr',
 'Ashland Ave & 50th St',
 'Western Ave & Monroe St',
 'Canal St & Madison St',
 'Calumet Ave & 51st St',
 'Dodge Ave & Mulford St',
 'Morgan St & 31st St',
 'Halsted St & Roosevelt Rd',
 'State St & Van Buren St',
 'Lincolnwood Dr & Central St',
 'N Green St & W Lake St',
 'Cannon Dr & Fullerton Ave',
 'Cottage Grove Ave & 71st St',
 'Lawndale Ave & 111th St',
 'Winchester (Ravenswood) Ave & Balmoral Ave',
 'Calumet Ave & 21st St',
 'Central Park Blvd & 5th Ave',
 'Halsted St & 56th St',
 'Fairfield Ave & Roosevelt Rd',
 'Winchester Ave & Elston Ave',
 'Bradley Park',
 'Sheffield Ave & Kingsbury St',
 'Michigan Ave & Pearson St',
 'Michigan Ave & 114th St',
 'Marshfield Ave & 44th St',
 'N Paulina St & Lincoln Ave',
 'Walsh Park',
 'Central Park Ave & Bloomingdale Ave',
 'Stave St & Armitage Ave',
 'LaSalle St & Illinois St',
 'Greenwood Ave & 97th St',
 'Desplaines St & Kinzie St',
 'Greenview Ave & Diversey Pkwy',
 'Halsted St & 63rd St',
 'Broadway & Barry Ave',
 'Western Ave & Leland Ave',
 'Wells St & 19th St',
 'Daley Center Plaza',
 'Lake Shore Dr & Ohio St',
 'Ashland Ave & Augusta Blvd',
 'Lincoln Ave & Roscoe St',
 'Eastlake Ter & Rogers Ave',
 'Damen Ave & Thomas St (Augusta Blvd)',
 'Milwaukee Ave & Grand Ave',
 'Wabash Ave & Wacker Pl',
 'Lake Shore Dr & North Blvd',
 'Ogden Ave & Race Ave',
 'Dayton St & North Ave',
 'Kingsbury St & Kinzie St',
 'Dearborn St & Monroe St',
 'Wells St & Walton St',
 'Aberdeen St & Jackson Blvd',
 'Larrabee St & Armitage Ave',
 'Wentworth Ave & 33rd St',
 'California Ave & 21st St',
 'Halsted St & 37th St',
 'Paulina St & 18th St',
 'Ashland Ave & 13th St',
 'Laramie Ave & Madison St',
 'Cottage Grove Ave & 43rd St',
 'Lakefront Trail & Wilson Ave',
 'MLK Jr Dr & Pershing Rd',
 'Burnham Harbor',
 'Ellis Ave & 60th St',
 'Ravenswood Ave & Lawrence Ave',
 'Broadway & Argyle St',
 'Lincoln Ave & Diversey Pkwy',
 'Central Ave & Harrison St',
 'Morgan St & 18th St',
 'MLK Jr Dr & 29th St',
 'Albany Ave & Bloomingdale Ave',
 'Woodlawn Ave & Lake Park Ave',
 'Indiana Ave & 40th St',
 'Clark St & Lunt Ave',
 'Knox Ave & Montrose Ave',
 'Clark St & Lincoln Ave',
 'Clark St & Winnemac Ave',
 'Leavitt St & North Ave',
 'Leavitt St & Lawrence Ave',
 'Southport Ave & Waveland Ave',
 'Lincoln Ave & Belle Plaine Ave',
 'Malcolm X College',
 'Franklin St & Adams St (Temp)',
 'Orleans St & Elm St',
 'Cornell Ave & Hyde Park Blvd',
 'N Southport Ave & W Newport Ave',
 'Clark St & Grace St',
 'Clark St & Leland Ave',
 'State St & 19th St',
 'Broadway & Wilson Ave',
 'Princeton Ave & Garfield Blvd',
 'Base - 2132 W Hubbard Warehouse',
 'Jeffery Blvd & 76th St',
 'Shore Dr & 55th St',
 'Sheridan Rd & Noyes St (NU)',
 'Sheridan Rd & Columbia Ave',
 'Bernard St & Elston Ave',
 'Elizabeth (May) St & Fulton St',
 'Wells St & Concord Ln',
 'Michigan Ave & Oak St',
 'Francisco Ave & Foster Ave',
 'Sheridan Rd & Loyola Ave',
 'Canal St & Jackson Blvd',
 'Western Ave & 28th St',
 'Western Ave & Congress Pkwy',
 'State St & 29th St',
 'Loomis St & Lexington St',
 'Leavitt St & Addison St',
 'Laramie Ave & Kinzie St',
 'Racine Ave & Fullerton Ave',
 'Kedzie Ave & Leland Ave',
 'Millennium Park',
 'Stony Island Ave & 82nd St',
 'Fairbanks Ct & Grand Ave',
 'Commercial Ave & 83rd St',
 'California Ave & 26th St',
 'South Shore Dr & 71st St',
 'Kedzie Ave & Harrison St',
 'Ravenswood Ave & Berteau Ave',
 'Ellis Ave & 58th St',
 'Kedzie Ave & 21st St',
 'Marquette Ave & 89th St',
 'State St & 79th St',
 'Lake Park Ave & 47th St',
 'Keystone Ave & Montrose Ave',
 'Cottage Grove Ave & 111th Pl',
 'Ashland Ave & Grace St',
 'Ewing Ave & Burnham Greenway',
 'Blue Island Ave & 18th St',
 'Wood St & Taylor St (Temp)',
 'Sheffield Ave & Wrightwood Ave',
 '63rd St Beach',
 'Halsted St & 78th St',
 'Western Ave & Granville Ave',
 'Cicero Ave & Flournoy St',
 'Austin Blvd & Chicago Ave',
 'Malcolm X College Vaccination Site',
 'Sawyer Ave & Irving Park Rd',
 'Clinton St & Tilden St',
 'South Shore Dr & 67th St',
 'Stony Island Ave & 75th St',
 'Wentworth Ave & Cermak Rd',
 'Eggleston Ave & 69th St',
 'Racine Ave & Wrightwood Ave',
 'Vernon Ave & 107th St',
 'Warren Park East',
 'Broadway & Wilson - Truman College Vaccination Site',
 'Damen Ave & Wellington Ave',
 'Halsted St & 18th St',
 'Carpenter St & Huron St',
 'Phillips Ave & 83rd St',
 'Elizabeth St & 47th St',
 'Dauphin Ave & 87th St',
 'Eberhart Ave & 91st St',
 'Wabash Ave & 87th St',
 'Loomis St & Archer Ave',
 'Clark St & Touhy Ave',
 'Wabash Ave & 16th St',
 'Glenwood Ave & Morse Ave',
 'Racine Ave & 61st St',
 'Cottage Grove Ave & 63rd St',
 'Franklin St & Illinois St',
 'Sheridan Rd & Greenleaf Ave',
 'Larrabee St & Webster Ave',
 'State St & 33rd St',
 'Adler Planetarium',
 'Dearborn St & Erie St',
 'Ashland Ave & Division St',
 'Kedzie Ave & Roosevelt Rd',
 'Shields Ave & 43rd St',
 'Kostner Ave & Lake St',
 'Eggleston Ave & 92nd St',
 'Morgan Ave & 14th Pl',
 'Ashland Ave & Pershing Rd',
 'California Ave & Montrose Ave',
 'California Ave & Division St',
 'Evans Ave & 75th St',
 'Sheridan Rd & Lawrence Ave',
 'Clarendon Ave & Gordon Ter',
 'Central St Metra',
 'Cottage Grove Ave & 78th St',
 'Prospect Sq & 91st St',
 'Cottage Grove Ave & 67th St',
 'Damen Ave & Melrose Ave',
 'Yates Blvd & 75th St',
 'Jeffery Blvd & 67th St',
 'Cicero Ave & Lake St',
 'W Washington Blvd & N Peoria St',
 'Kedzie Ave & 110th St',
 'Rhodes Ave & 71st St',
 'Dauphin Ave & 103rd St',
 'Field Museum',
 'Pulaski Rd & Congress Pkwy',
 'Ravenswood Ave & Irving Park Rd',
 'Hale Ave & 107th St',
 'W Oakdale Ave & N Broadway',
 'State St & 54th St',
 'Torrence Ave & 106th St',
 'St. Louis Ave & Fullerton Ave',
 'Ashland Ave & McDowell Ave',
 'Central Park Ave & North Ave',
 'Morgan St & Pershing Rd',
 'Lake Park Ave & 53rd St',
 'Kenton Ave & Madison St',
 'Cottage Grove Ave & 51st St',
 'Halsted St & 96th St',
 'Wentworth Ave & 104th St',
 'Vincennes Ave & 104th St',
 'Leavitt St & Archer Ave',
 'Chicago Ave & Sheridan Rd',
 'Dorchester Ave & 63rd St',
 'Stony Island Ave & 71st St',
 'Milwaukee Ave & Cuyler Ave',
 'State St & 95th St',
 'Cornell Dr & Hayes Dr',
 'Stony Island Ave & 64th St',
 'Ellis Ave & 83rd St',
 'Wolcott Ave & Polk St',
 'Clark St & Wellington Ave',
 'Sheridan Rd & Buena Ave',
 'Prairie Ave & Garfield Blvd',
 'Sedgwick St & Schiller St',
 'State St & Pershing Rd',
 'Damen Ave & 51st St',
 'Austin Blvd & Lake St',
 'Avondale Ave & Irving Park Rd',
 'Eckhart Park',
 'Stony Island Ave & South Chicago Ave',
 'Union Ave & Root St',
 'Sacramento Blvd & Franklin Blvd',
 'Central Park Ave & 24th St',
 'Baltimore Ave & 87th St',
 'Ogden Ave & Congress Pkwy',
 'Kostner Ave & Adams St',
 'N Sheffield Ave & W Wellington Ave',
 'Austin Blvd & Madison St',
 'Ashland Ave & Garfield Blvd',
 'Hermitage Ave & Polk St',
 'Michigan Ave & 71st St',
 'Halsted St & 51st St',
 'East End Ave & 87th St',
 'Clyde Ave & 87th St',
 'Bennett Ave & 79th St',
 'Indiana Ave & 103rd St',
 'Rainbow Beach',
 'Halsted St & 73rd St',
 'Phillips Ave & 79th St',
 'W Armitage Ave & N Sheffield Ave',
 'Green St & Randolph St',
 'Desplaines St & Randolph St',
 'Franklin St & Lake St',
 'Marshfield Ave & Cortland St',
 'Cottage Grove Ave & 47th St',
 'Stewart Ave & 63rd St',
 'Sheffield Ave & Wellington Ave',
 'Broadway & Ridge Ave',
 'Broadway & Berwyn Ave',
 'Western Ave & Howard St',
 'Avers Ave & Belmont Ave',
 'Clinton St & Washington Blvd',
 'Broadway & Waveland Ave',
 'Budlong Woods Library',
 'Racine Ave & Randolph St',
 'Damen Ave & Coulter St',
 'Wabash Ave & Adams St',
 'Rush St & Cedar St',
 'Pine Grove Ave & Waveland Ave',
 'Western Ave & Winnebago Ave',
 'Shields Ave & 28th Pl',
 'Calumet Ave & 18th St',
 'Sheridan Rd & Irving Park Rd',
 'Burling St & Diversey Pkwy',
 'Seeley Ave & Roscoe St',
 'Kedzie Ave & Foster Ave',
 'State St & 123rd St',
 'Avenue O & 118th St',
 'Michigan Ave & Washington St',
 'Prairie Ave & 43rd St',
 'May St & Taylor St',
 'Clybourn Ave & Division St',
 'Ogden Ave & Roosevelt Rd',
 'Lakefront Trail & Bryn Mawr Ave',
 'Lake Shore Dr & Monroe St',
 'Southport Ave & Clybourn Ave',
 'Elmwood Ave & Austin St',
 'Seeley Ave & Garfield Blvd',
 'Homewood Ave & 115th St',
 'Stetson Ave & South Water St',
 'Wallace St & 35th St',
 'Rockwell St & Eastwood Ave',
 'Orleans St & Chestnut St (NEXT Apts)',
 'Walden Pkwy & 100th St',
 'Halsted St & Wrightwood Ave',
 'South Shore Dr & 74th St',
 'Sheffield Ave & Waveland Ave',
 'Southport Ave & Belmont Ave',
 'Leavitt St & Division St',
 'Michigan Ave & Madison St',
 'Dusable Harbor',
 'Dearborn St & Van Buren St',
 'Michigan Ave & 18th St',
 'Broadway & Granville Ave',
 'Big Marsh Park',
 'Manor Ave & Leland Ave',
 'Delano Ct & Roosevelt Rd',
 'Oakley Ave & Touhy Ave',
 'Aberdeen St & Randolph St',
 'Field Blvd & South Water St',
 'Canal St & Monroe St',
 'Ashland Ave & Chicago Ave',
 'Central Ave & Chicago Ave',
 'Stewart Ave & 83rd St',
 'Eberhart Ave & 61st St',
 'MLK Jr Dr & 83rd St',
 'Talman Ave & Addison St',
 'California Ave & 23rd Pl',
 'Western Ave & Roscoe St',
 'Vernon Ave & 75th St',
 'Stony Island Ave & 90th St',
 'Clark St & Newport St',
 'Laflin St & Cullerton St',
 'N Carpenter St & W Lake St',
 'Dodge Ave & Church St',
 'Western Blvd & 48th Pl',
 'Sheridan Rd & Montrose Ave',
 'Clark St & North Ave',
 'MLK Jr Dr & 47th St',
 'Woodlawn Ave & 75th St',
 'Blackstone Ave & Hyde Park Blvd',
 'Damen Ave & Pershing Rd',
 'Greenwood Ave & 91st St',
 'Kilbourn Ave & Milwaukee Ave',
 'Cherry Ave & Blackhawk St',
 'Kedzie Ave & 24th St',
 'Karlov Ave & Madison St',
 'Stony Island Ave & 67th St',
 'Loomis Blvd & 84th St',
 'Cottage Grove Ave & 83rd St',
 'Pulaski Rd & Lake St',
 'Avenue O & 134th St',
 'Elizabeth St & 92nd St',
 'Western Ave & 111th St',
 'Ada St & 113th St',
 'Calumet Ave & 71st St',
 'Halsted St & 111th St',
 'Halsted & 63rd - Kennedy-King Vaccination Site',
 'Ashland Ave & 74th St',
 'Western & 28th - Velasquez Institute Vaccination Site',
 'Constance Ave & 95th St',
 'Western Ave & 104th St',
 'Summit Ave & 86th St',
 'Cicero Ave & Quincy St',
 'Major Taylor Trail & 124th St',
 'Chicago State University',
 'Ashland Ave & 66th St',
 'Vincennes Ave & 75th St',
 'May St & 69th St',
 'State St & 76th St',
 'Wabash Ave & 83rd St',
 'Elizabeth St & 59th St',
 'Houston Ave & 92nd St',
 'Damen Ave & Wabansia Ave',
 'Marshfield Ave & 59th St',
 'Eberhart Ave & 131st St',
 'N Damen Ave & W Wabansia St',
 'S Wentworth Ave & W 111th St',
 'Torrence Ave & 126th Pl',
 'Laramie Ave & Gladys Ave',
 'Damen Ave & 59th St',
 'N Hampden Ct & W Diversey Ave',
 'Major Taylor Trail & 115th St',
 'Commercial Ave & 130th St',
 'Halsted St & 59th St',
 'Exchange Ave & 79th St',
 'Altgeld Gardens',
 'Halsted St & 104th St',
 'Hampden Ct & Diversey Ave',
 'Kedzie Ave & 104th St',
 'Avenue L & 114th St',
 'South Chicago Ave & 83rd St',
 'Throop St & 52nd St',
 'Calumet Park',
 'Carpenter St & 63rd St',
 'Elston Ave & Cortland St',
 'Racine Ave & 65th St',
 'Loomis St & 89th St',
 'Woodlawn & 103rd - Olive Harvey Vaccination Site',
 'W 103rd St & S Avers Ave',
 'South Chicago Ave & Elliot Ave',
 'N Clark St & W Elm St',
 'WEST CHI-WATSON',
 'Yates Blvd & 93rd St',
 'Maryland Ave & 104th St',
 'S Aberdeen St & W 106th St',
 'Lyft Driver Center Private Rack',
 'Cicero Ave & Grace St',
 'DuSable Lake Shore Dr & Belmont Ave',
 'DuSable Lake Shore Dr & Monroe St',
 'DuSable Lake Shore Dr & North Blvd',
 'DuSable Lake Shore Dr & Ohio St',
 'DuSable Lake Shore Dr & Wellington Ave',
 'DuSable Lake Shore Dr & Diversey Pkwy',
 'Kildare Ave & 26th St',
 'Tripp Ave & 31st St',
 'Halsted St & 18th St (Temp)',
 'Lawndale Ave & 16th St',
 'Spaulding Ave & 16th St',
 'Homan Ave & Fillmore St',
 'Lavergne & Fullerton',
 'DIVVY CASSETTE REPAIR MOBILE STATION',
 'Kilpatrick Ave & Parker Ave',
 'Central Park Ave & Douglas Blvd',
 'Lamon Ave & Belmont Ave',
 'Long Ave & Belmont Ave',
 'Meade Ave & Diversey Ave',
 'Long Ave & Belden Ave',
 'Tripp Ave & 15th St',
 'Mason Ave & Belmont Ave',
 'Parkside Ave & Armitage Ave',
 'S Michigan Ave & E 118th St',
 'Keeler Ave & Roosevelt Rd',
 'Pulaski Rd & 21st St',
 'Central Ave & Parker Ave',
 'Lawndale Ave & 30th St',
 'Meade Ave & Addison St',
 'Kilbourn & Roscoe',
 'Mulligan Ave & Wellington Ave',
 'Lockwood Ave & Wrightwood Ave',
 'Plainfield & Irving Park',
 'Narragansett Ave & School St',
 'Central Ave & Roscoe St',
 '351',
 'Long & Irving Park',
 'Komensky Ave & 31st St',
 'Kilbourn & Belden',
 'Kostner Ave & Wrightwood Ave',
 'North Ave & New England Ave',
 'Harding Ave & 26th St',
 'Harlem Ave & Grace St',
 'Oketo Ave & Addison St',
 'Roscoe & Harlem',
 'Lamon Ave & Armitage Ave',
 'Sayre & Diversey',
 'Orange & Addison',
 'Harlem & Irving Park',
 'Menard Ave & Division St',
 'Narragansett & Wrightwood',
 'Lamon Ave & Chicago Ave',
 'Rockwell St & Archer Ave',
 'Kedzie Ave & 52nd St',
 'Panama Ave & Grace St',
 'Major Ave & Bloomingdale Ave',
 'Nordica & Medill',
 'Narragansett & Irving Park',
 'Western Ave & 62nd St',
 'Doty Ave & 111th St',
 'Kenosha & Wellington',
 'Oak Park & Wellington',
 'Kedzie Ave & 48th Pl',
 'Narragansett & McLean',
 'Archer Ave & 43rd St',
 'Monticello Ave & Chicago Ave',
 'Richmond St & 59th St',
 'Bloomingdale Ave & Harlem Ave',
 'Hoyne Ave & 34th St',
 'Long Ave & North Ave',
 'Lavergne Ave & Division St',
 'St Louis Ave & Norman Bobbins Ave',
 'Kildare Ave & Chicago Ave',
 'Maplewood Ave & 59th St',
 'Kildare Ave & Division St',
 'Ping Tom Park (East)',
 'Sacramento Ave & Pershing Rd',
 'California Ave & 36th St',
 'Leamington Ave & Hirsch St',
 'Western Ave & Gunnison St',
 'Rockwell St & 57th St',
 'Damen Ave & 33rd St',
 'Kildare Ave & 55th St',
 'N Damen Ave & W Chicago Ave',
 'Kostner Ave & 63rd St',
 'Springfield Ave & 47th St',
 'Pulaski Rd & 51st St',
 'Lavergne Ave & 46th St',
 'Fairfield Ave & 44th St',
 'Tripp Ave & 65th St',
 'Troy St & Grace St',
 'Pulaski Rd & 60th St',
 'Komensky Ave & 55th St',
 'Kedzie Ave & 57th St',
 'Central Park & Augusta Blvd',
 'Lawler Ave & 50th St',
 'Central Park Ave & Ohio St',
 'Albany Ave & Belmont Ave',
 'Milwaukee Ave & Fullerton Ave',
 'Spaulding Ave & 63rd St',
 'Artesian Ave & 55th St',
 'Pulaski & Ann Lurie Pl',
 'Indiana Ave & 133rd St',
 'Lincoln Ave & Balmoral Ave',
 'Shabbona Park',
 'California Ave & Marquette Rd',
 'Campbell Ave & 51st St',
 'Hamlin Ave & 62nd Pl',
 'Kenneth Ave & 50th St',
 'Whipple St & Irving Park Rd',
 'St Louis Ave & 59th St',
 'Kilpatrick Ave & Grand Ave',
 'California Ave & Winona St',
 'Whipple/Irving Park',
 'Kildare Ave & 47th St',
 'Western Ave & Grace St',
 'Rockwell St & Fletcher St',
 'Washtenaw Ave & Peterson Ave',
 'Elston Ave & George St',
 'Lincoln Ave & Peterson Ave',
 'Francisco Ave & Bloomingdale Ave',
 'N Shore Channel Trail & Argyle Ave',
 'Kedzie Ave & George St',
 'Augusta Blvd & Laramie Ave',
 'Rockwell St & Cermak Rd',
 'Francisco Ave & Montrose Ave',
 'Virginia Ave & Catalpa Ave',
 'Whipple St & Roosevelt Rd',
 'Richmond St & Lincoln Ave',
 'Elston Ave & Henderson St',
 'Sacramento Blvd & Fulton Ave',
 'Sacramento Blvd & Polk St',
 'Throop/Hastings Mobile Station',
 'Menard Ave & North Ave',
 'Sacramento Ave & 25th St',
 'Francisco Ave & Hollywood Ave',
 'Mozart St & Jackson Blvd',
 'Washtenaw Ave & Madison St',
 'Western Ave & Ardmore Ave',
 'Sacramento Blvd & Addison St',
 'Western Ave & Lake St',
 'Washtenaw Ave & Polk St',
 'Francisco Ave & 47th St',
 'California Ave & 29th St',
 'State St & Chicago Ave',
 'McClurg Ct & Ohio St',
 'Olive Harvey College']

In [7]:
len(allSt21)

847

In [8]:
removed21 = list(set(allSt21) - set(allSt22))
len(removed21)

50

In [9]:
added22 = list(set(allSt22) - set(allSt21))
added22.pop(0)

nan

In [10]:
len(added22)

10

# Plotting all added stations of 2022

In [11]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [12]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [13]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [14]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [15]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [16]:
#Adding co-ordinates to all the stations that are added this year
n22 = {k: locations[k] for k in locations.keys() & set(added22)}

#printing out the first item in the dictionary to check the key-value pair
list(n22.items())[0]

('Fullerton & Monitor', {'lat': 41.9237918, 'long': -87.7723116})

In [17]:
#Converting the dictionary into dataframe
st22 = pd.DataFrame.from_dict(n22, orient='index')

In [18]:
#Resetting the index
st22.reset_index(inplace=True)

#Renaming the index column to Station name
st22 = st22.rename(columns = {'index':'Station Name'})

In [19]:
#Adding a column with values of color for every station
st22['color'] = st22.apply(lambda row: 'purple', axis=1)

#printing to check the structure of the dataframe
st22.head(1)

,Station Name,lat,long,color
0,Fullerton & Monitor,41.923792,-87.772312,purple


# Location Marker map for interactivity purpose (Station Name)

In [20]:
#setting marker as the color mentioned in the color column of the dataframe
marker = st22.loc[st22['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2022
## Saving the map for converting into gif

In [21]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2022')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "red",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st22.apply(plotDot, axis = 1)

m

In [22]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [23]:
m.save('Gif_circlemarker/2022.html')

In [24]:
with fileinput.FileInput('Gif_circlemarker/2022.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [25]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2022.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2022.png')
browser.quit()

In [26]:
image = Image.open('ss2022.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2022.png')

# Heatmap 2022
## Saving the map for converting into gif

In [27]:
numlist = st22.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2022')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [28]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [29]:
m.save('Gif_heatmap/2022.html')

In [30]:
with fileinput.FileInput('Gif_heatmap/2022.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [31]:
1# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2022.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2022.png')
browser.quit()

In [33]:
image = Image.open('ss2022.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2022.png')